
# NVDA Momentum Alignment Trace (ipynb)

This notebook loads **NVDA** prices from your DuckDB warehouse and shows, for each output row:

- `t_date` (the row's date `t`)
- `numerator_date / numerator_price` = price at `t - skip`
- `denominator_date / denominator_price` = price at `t - skip - lookback`
- `value_check` = `numerator_price / denominator_price - 1`

Use this to visually verify the index math for momentum \( P[t-skip] / P[t-skip-lookback] - 1 \).


In [3]:

# === Parameters ===
DB_PATH = "../../../data/warehouse/data.duckdb"  # Path to your DuckDB file
LOOKBACK_MONTHS = 12
SKIP_MONTHS = 1
TRADING_DAYS_PER_MONTH = 21
ROWS_TO_SHOW = 60  # how many rows from the end to display
PRICE_COL = "adj_close"
SYMBOL = "NVDA"


In [4]:

import duckdb
import pandas as pd
import numpy as np
pd.set_option("display.max_rows", 200)


In [5]:

def load_symbol_prices(con, symbol: str, price_col: str = "adj_close") -> pd.DataFrame:
    """Load price series for a given symbol using the `securities` mapping.
    Returns columns: security_id, trade_date, adj_close
    """
    row = con.execute("SELECT security_id FROM securities WHERE symbol = ?", [symbol]).fetchone()
    if row is None:
        raise RuntimeError(f"Could not find {symbol} in `securities` table.")
    sid = row[0]

    df = con.execute(f'''
        SELECT security_id, trade_date, {price_col} AS adj_close
        FROM prices
        WHERE security_id = ?
        ORDER BY trade_date
    ''', [sid]).fetchdf()

    if df.empty:
        raise RuntimeError(f"No {symbol} price data found in `prices`.")
    df["trade_date"] = pd.to_datetime(df["trade_date"])
    df = df.dropna(subset=["adj_close"]).reset_index(drop=True)
    return df


In [6]:

def build_trace(df: pd.DataFrame, lookback_months: int = LOOKBACK_MONTHS, skip_months: int = SKIP_MONTHS, trading_days_per_month: int = TRADING_DAYS_PER_MONTH) -> pd.DataFrame:
    """Build a trace that makes numerator/denominator alignment explicit.
    momentum(t) = P[t - skip_days] / P[t - skip_days - lookback_days] - 1
    """
    lookback_days = lookback_months * trading_days_per_month
    skip_days = skip_months * trading_days_per_month

    trace = df.copy()
    # numerator at (t - skip)
    trace["numerator_date"] = trace["trade_date"].shift(skip_days)
    trace["numerator_price"] = trace["adj_close"].shift(skip_days)
    # denominator at (t - skip - lookback)
    trace["denominator_date"] = trace["trade_date"].shift(skip_days + lookback_days)
    trace["denominator_price"] = trace["adj_close"].shift(skip_days + lookback_days)

    trace["value_check"] = (trace["numerator_price"] / trace["denominator_price"]) - 1.0

    # Require full history
    trace = trace.dropna(subset=["numerator_price", "denominator_price"]).copy()
    trace["factor_name"] = f"momentum_{lookback_months}m_skip_{skip_months}m"
    trace = trace.rename(columns={"trade_date": "t_date"})

    cols = [
        "t_date",
        "numerator_date", "numerator_price",
        "denominator_date", "denominator_price",
        "value_check", "factor_name"
    ]
    return trace[cols]


In [7]:

def compare_with_vectorized(trace_df: pd.DataFrame, df_raw: pd.DataFrame, lookback_months: int, skip_months: int, trading_days_per_month: int = 21) -> pd.DataFrame:
    """Recompute momentum via a vectorized core and compare with value_check."""
    lookback = lookback_months * trading_days_per_month
    skip = skip_months * trading_days_per_month

    arr = df_raw["adj_close"].to_numpy(dtype=float)
    out = np.full_like(arr, np.nan, dtype=float)
    n = arr.shape[0]
    if n > lookback + skip:
        # FIXED slices:
        num = arr[lookback : n - skip]
        den = arr[: n - (lookback + skip)]
        valid = (den != 0) & np.isfinite(den) & np.isfinite(num)
        tmp = np.full(n - (lookback + skip), np.nan, dtype=float)
        tmp[valid] = num[valid] / den[valid] - 1.0
        out[lookback + skip :] = tmp

    df_func = df_raw.copy()
    df_func["value_func"] = out
    df_func = df_func.rename(columns={"trade_date": "t_date"})
    merged = trace_df.merge(df_func[["t_date", "value_func"]], on="t_date", how="left")

    diff = np.nanmax(np.abs(merged["value_check"] - merged["value_func"])) if len(merged) else np.nan
    print(f"[Compare] max |value_check - value_func| = {diff:.12f}")
    return merged


In [8]:

# Connect, load NVDA, build trace, compare, and show tail
con = duckdb.connect(DB_PATH)
df_nvda = load_symbol_prices(con, SYMBOL, price_col=PRICE_COL)
trace = build_trace(df_nvda, LOOKBACK_MONTHS, SKIP_MONTHS, TRADING_DAYS_PER_MONTH)
traced = compare_with_vectorized(trace, df_nvda, LOOKBACK_MONTHS, SKIP_MONTHS, TRADING_DAYS_PER_MONTH)

# Show last N rows for manual verification
traced.tail(ROWS_TO_SHOW)



[Compare] max |value_check - value_func| = 0.000000000000


,t_date,numerator_date,numerator_price,denominator_date,denominator_price,value_check,factor_name,value_func
1898,2025-08-22,2025-07-24,173.73,2024-07-22,123.49,0.406835,momentum_12m_skip_1m,0.406835
1899,2025-08-25,2025-07-25,173.49,2024-07-23,122.54,0.415783,momentum_12m_skip_1m,0.415783
1900,2025-08-26,2025-07-28,176.74,2024-07-24,114.21,0.547500,momentum_12m_skip_1m,0.547500
1901,2025-08-27,2025-07-29,175.50,2024-07-25,112.24,0.563614,momentum_12m_skip_1m,0.563614
1902,2025-08-28,2025-07-30,179.26,2024-07-26,113.02,0.586091,momentum_12m_skip_1m,0.586091
1903,2025-08-29,2025-07-31,177.86,2024-07-29,111.55,0.594442,momentum_12m_skip_1m,0.594442
1904,2025-09-02,2025-08-01,173.71,2024-07-30,103.69,0.675282,momentum_12m_skip_1m,0.675282
1905,2025-09-03,2025-08-04,179.99,2024-07-31,116.98,0.538639,momentum_12m_skip_1m,0.538639
1906,2025-09-04,2025-08-05,178.25,2024-08-01,109.17,0.632775,momentum_12m_skip_1m,0.632775
1907,2025-09-05,2025-08-06,179.41,2024-08-02,107.23,0.673133,momentum_12m_skip_1m,0.673133


In [9]:
con.close()